Iñigo Martin Melero

## Analisis SMOTE Amat

### 0) Librerias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from collections import Counter

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import GradientBoostingClassifier

np.random.seed(51)

### 1) Preprocesamiento

In [2]:
df1=pd.read_csv('Bancarrota_Total_nd.csv',encoding = 'unicode_escape')
print(df1)

        ID  Coef1_Springate  Coef2_Springate  Coef3_Springate  \
0        1           0.9525           0.0062           0.0054   
1        2           0.6894           0.1641           0.3116   
2        3           0.8671          -0.0479          -0.5388   
3        4          -0.0034           0.0296          -0.2114   
4        5           0.2916           0.0341           0.0534   
...    ...              ...              ...              ...   
1440  1502          -0.0555          -0.3837          -0.3051   
1441  1503           0.0061          -0.4382          -1.0570   
1442  1505           0.8625         -24.6623          -6.1772   
1443  1511          -2.2275          -0.0105          -0.0031   
1444  1512          -0.9579          -0.1787          -0.0904   

      Coef4_Springate  Coef1_Amat  Coef2_Amat  Coef3_Amat  Coef4_Amat  \
0              3.9499      1.0056      0.0064     -0.0150     -2.3367   
1              2.8739      1.7204      0.4234      0.1196      0.2824   


In [3]:
df1.dtypes

ID                   int64
Coef1_Springate    float64
Coef2_Springate    float64
Coef3_Springate    float64
Coef4_Springate    float64
Coef1_Amat         float64
Coef2_Amat         float64
Coef3_Amat         float64
Coef4_Amat         float64
Resultado            int64
dtype: object

In [4]:
porcentaje_train=0.8

In [5]:
limitedf1=round(porcentaje_train*len(df1))
limitedf1

1156

In [6]:
df1_train=df1[:limitedf1]
df1_train

,ID,Coef1_Springate,Coef2_Springate,Coef3_Springate,Coef4_Springate,Coef1_Amat,Coef2_Amat,Coef3_Amat,Coef4_Amat,Resultado
0,1,0.9525,0.0062,0.0054,3.9499,1.0056,0.0064,-0.0150,-2.3367,1
1,2,0.6894,0.1641,0.3116,2.8739,1.7204,0.4234,0.1196,0.2824,1
2,3,0.8671,-0.0479,-0.5388,2.0795,0.4260,-1.4245,-1.1645,0.8174,1
3,4,-0.0034,0.0296,-0.2114,0.6265,0.4756,0.2109,-0.1652,-0.7833,1
4,5,0.2916,0.0341,0.0534,0.3098,0.9662,0.3709,0.0299,0.0807,1
...,...,...,...,...,...,...,...,...,...,...
1151,1169,-0.1073,-0.2998,-1.4365,0.5178,2.4002,0.2653,-0.2303,-0.8683,1
1152,1170,0.1210,0.1520,0.4226,1.4999,2.8427,0.3009,0.0441,0.1465,1
1153,1171,-3.6402,-42.3286,-0.9917,19.7943,0.0231,-42.2713,-42.9121,1.0152,1
1154,1172,0.6049,-0.0977,-0.2737,0.4939,1.7736,0.6123,-0.0977,-0.1596,1


In [7]:
df1_test=df1[limitedf1:]
df1_test

,ID,Coef1_Springate,Coef2_Springate,Coef3_Springate,Coef4_Springate,Coef1_Amat,Coef2_Amat,Coef3_Amat,Coef4_Amat,Resultado
1156,1174,0.4748,0.0758,0.2129,3.4304,2.8091,0.6440,0.0568,0.0883,1
1157,1176,-0.1940,-0.1275,-0.4370,0.2235,0.5615,0.0590,-0.1274,-2.1603,1
1158,1177,0.0978,0.0282,0.1630,0.4326,1.5040,0.7844,0.0209,0.0267,1
1159,1178,0.0217,-0.0145,-0.6856,0.0002,12.5930,0.3185,-0.0251,-0.0787,1
1160,1179,0.0953,-0.9579,-1.9177,2.7373,0.1924,-0.6964,-0.8234,1.1824,1
...,...,...,...,...,...,...,...,...,...,...
1440,1502,-0.0555,-0.3837,-0.3051,0.0354,0.7084,-0.2591,-0.3842,1.4825,1
1441,1503,0.0061,-0.4382,-1.0570,0.0156,0.7955,-1.6397,-0.4123,0.2515,1
1442,1505,0.8625,-24.6623,-6.1772,0.1219,0.2487,-9.1522,-24.8344,2.7135,1
1443,1511,-2.2275,-0.0105,-0.0031,0.0000,0.2941,-2.3456,-0.0105,0.0045,1


In [8]:
df2=pd.read_csv('Activa_Total_nd.csv',encoding = 'unicode_escape')
print(df2)

            ID  Coef1_Springate  Coef2_Springate  Coef3_Springate  \
0            1          -0.1786           0.0810           0.2086   
1            2           0.3473           0.0039           0.1439   
2            3           0.4718           0.0385           0.0752   
3            4           0.1373           0.0490           0.0493   
4            5           0.0656           0.0826           0.5767   
...        ...              ...              ...              ...   
675939  732517           0.1232          -0.1647          -5.9479   
675940  732532           0.1989          -2.1463          -0.7942   
675941  732550          -0.0927           0.0000           0.0208   
675942  732570           0.6813          -0.0064          -0.0240   
675943  732590           0.4244          -0.0416          -0.0402   

        Coef4_Springate  Coef1_Amat  Coef2_Amat  Coef3_Amat  Coef4_Amat  \
0                2.3157      0.9028      0.6267      0.0598      0.0954   
1                1.89

In [9]:
df2.dtypes

ID                   int64
Coef1_Springate    float64
Coef2_Springate    float64
Coef3_Springate    float64
Coef4_Springate    float64
Coef1_Amat         float64
Coef2_Amat         float64
Coef3_Amat         float64
Coef4_Amat         float64
Resultado            int64
dtype: object

In [10]:
limitedf2=round(porcentaje_train*len(df2))
limitedf2

540755

In [11]:
df2_train=df2[:limitedf2]
df2_train

,ID,Coef1_Springate,Coef2_Springate,Coef3_Springate,Coef4_Springate,Coef1_Amat,Coef2_Amat,Coef3_Amat,Coef4_Amat,Resultado
0,1,-0.1786,0.0810,0.2086,2.3157,0.9028,0.6267,0.0598,0.0954,2
1,2,0.3473,0.0039,0.1439,1.8919,1.0684,0.2626,0.0774,0.2946,2
2,3,0.4718,0.0385,0.0752,2.1073,1.4550,0.4008,0.0199,0.0498,2
3,4,0.1373,0.0490,0.0493,11.8888,1.0009,0.0042,0.0366,8.7045,2
4,5,0.0656,0.0826,0.5767,0.6623,2.1201,0.8463,0.0806,0.0953,2
...,...,...,...,...,...,...,...,...,...,...
540750,560455,0.0157,0.0146,0.3263,0.0371,1.2912,0.5212,0.0081,0.0155,2
540751,560456,0.0258,-0.0088,-0.7119,0.1700,2.4988,0.0089,-0.0080,-0.8908,2
540752,560457,0.0061,-0.0209,15.1405,0.0140,230.5532,0.9513,0.0223,0.0234,2
540753,560458,0.5717,0.2600,1.4611,0.7148,4.6067,0.5313,0.2090,0.3935,2


In [12]:
df2_test=df2[limitedf2:]
df2_test

,ID,Coef1_Springate,Coef2_Springate,Coef3_Springate,Coef4_Springate,Coef1_Amat,Coef2_Amat,Coef3_Amat,Coef4_Amat,Resultado
540755,560460,-0.1669,0.0236,0.0120,0.6623,1.0110,-0.3342,0.0078,-0.0232,2
540756,560461,0.5826,-0.3426,-0.2562,1.9967,0.7480,-0.3370,-0.3426,1.0165,2
540757,560462,0.1177,-0.0301,-0.5879,0.4660,1.4747,-0.1189,-0.0450,0.3780,2
540758,560463,0.1464,0.3508,1.2304,0.5097,3.2814,0.7042,0.3508,0.4981,2
540759,560464,0.2660,0.0007,-0.0170,1.1911,0.4078,0.0066,-0.0127,-1.9132,2
...,...,...,...,...,...,...,...,...,...,...
675939,732517,0.1232,-0.1647,-5.9479,0.0000,9.4479,0.9574,-0.1649,-0.1722,2
675940,732532,0.1989,-2.1463,-0.7942,0.0000,0.1321,-1.7025,-1.6097,0.9455,2
675941,732550,-0.0927,0.0000,0.0208,0.0000,1.0615,0.4367,0.0117,0.0269,2
675942,732570,0.6813,-0.0064,-0.0240,0.0000,3.6889,0.7320,0.0025,0.0034,2


### 2) Preparacion datos para ML

In [13]:
df3_train=pd.concat([df1_train, df2_train])
X_train=df3_train.drop(columns=["Resultado","ID"])
Y_train=df3_train.drop(columns=["ID","Coef1_Amat","Coef2_Amat","Coef3_Amat","Coef4_Amat","Coef1_Springate","Coef2_Springate","Coef3_Springate","Coef4_Springate"])

In [14]:
sm=SMOTE(random_state=42)
X_SMOTE,Y_SMOTE=sm.fit_resample(X_train,Y_train)
df3_train_SMOTE=pd.concat([X_SMOTE, Y_SMOTE],axis=1)
df3_train_SMOTE = df3_train_SMOTE.sample(frac=1).reset_index(drop=True)
df3_train_SMOTE

,Coef1_Springate,Coef2_Springate,Coef3_Springate,Coef4_Springate,Coef1_Amat,Coef2_Amat,Coef3_Amat,Coef4_Amat,Resultado
0,-0.003900,0.023700,0.030900,0.981600,1.537100,-0.125000,0.007300,-0.058500,2
1,-0.036800,0.192000,0.124100,2.056700,0.403700,-0.547600,0.144000,-0.263000,2
2,0.206859,-0.122366,-0.351874,0.882378,0.852863,0.398709,-0.114246,-0.291273,1
3,0.056200,-0.017300,-0.112400,0.082200,1.718300,0.732900,-0.016200,-0.022100,2
4,-0.441470,-2.041330,-0.853690,0.485356,0.102403,-2.265369,-2.275014,0.995709,1
...,...,...,...,...,...,...,...,...,...
1081505,0.523600,-0.013600,-0.031800,0.940900,1.319600,0.124700,-0.015900,-0.127600,2
1081506,0.161500,0.103600,0.299100,1.730700,1.549800,0.301100,0.059900,0.198900,2
1081507,0.778800,0.002800,0.004000,1.459800,1.258300,0.299600,0.001800,0.006100,2
1081508,0.036000,0.014900,9.598900,0.038800,6.931500,0.814000,0.196600,0.241500,2


In [15]:
df3_test=pd.concat([df1_test, df2_test])
X_test=df3_test.drop(columns=["Resultado","ID"])
Y_test=df3_test.drop(columns=["ID","Coef1_Amat","Coef2_Amat","Coef3_Amat","Coef4_Amat","Coef1_Springate","Coef2_Springate","Coef3_Springate","Coef4_Springate"])

In [16]:
df3_test_SMOTE=pd.concat([X_test, Y_test],axis=1)

In [17]:
df3_total_SMOTE=pd.concat([df3_train_SMOTE, df3_test_SMOTE])
df3_total_SMOTE

,Coef1_Springate,Coef2_Springate,Coef3_Springate,Coef4_Springate,Coef1_Amat,Coef2_Amat,Coef3_Amat,Coef4_Amat,Resultado
0,-0.003900,0.023700,0.030900,0.981600,1.537100,-0.125000,0.007300,-0.058500,2
1,-0.036800,0.192000,0.124100,2.056700,0.403700,-0.547600,0.144000,-0.263000,2
2,0.206859,-0.122366,-0.351874,0.882378,0.852863,0.398709,-0.114246,-0.291273,1
3,0.056200,-0.017300,-0.112400,0.082200,1.718300,0.732900,-0.016200,-0.022100,2
4,-0.441470,-2.041330,-0.853690,0.485356,0.102403,-2.265369,-2.275014,0.995709,1
...,...,...,...,...,...,...,...,...,...
675939,0.123200,-0.164700,-5.947900,0.000000,9.447900,0.957400,-0.164900,-0.172200,2
675940,0.198900,-2.146300,-0.794200,0.000000,0.132100,-1.702500,-1.609700,0.945500,2
675941,-0.092700,0.000000,0.020800,0.000000,1.061500,0.436700,0.011700,0.026900,2
675942,0.681300,-0.006400,-0.024000,0.000000,3.688900,0.732000,0.002500,0.003400,2


In [18]:
X_train_SMOTE=df3_train_SMOTE.drop(columns=["Resultado"])
Y_train_SMOTE=df3_train_SMOTE.drop(columns=["Coef1_Amat","Coef2_Amat","Coef3_Amat","Coef4_Amat","Coef1_Springate","Coef2_Springate","Coef3_Springate","Coef4_Springate"])
X_test_SMOTE=df3_test_SMOTE.drop(columns=["Resultado"])
Y_test_SMOTE=df3_test_SMOTE.drop(columns=["Coef1_Amat","Coef2_Amat","Coef3_Amat","Coef4_Amat","Coef1_Springate","Coef2_Springate","Coef3_Springate","Coef4_Springate"])

### 3.1) Valores Test de Springate

Calculamos si está comprometida la situación de la empresa según Amat.

Su formula es:

Z = - 0.862 + 1.03 x Coef1 + 3.07 x Coef2 + 0.66 x Coef3 + 0.4 x Coef4

Z positivo empresa bien, Z negativo empresa mal.

Cogemos otro Dataframe, y multiplicamos primero y luego sumamos.

In [19]:
formula_Spr=X_test_SMOTE
formula_Spr=formula_Spr.drop(columns=["Coef1_Amat","Coef2_Amat","Coef3_Amat","Coef4_Amat"])
formula_Spr.loc[:,'Coef1_Springate'] *= 1.03
formula_Spr.loc[:,'Coef2_Springate'] *= 3.07
formula_Spr.loc[:,'Coef3_Springate'] *= 0.66
formula_Spr.loc[:,'Coef4_Springate'] *= 0.4
formula_Spr["constant"]=-0.862
formula_Spr["Z"] = formula_Spr.sum(axis=1)

formula_Spr["Y_pred_Springate"] = 2
formula_Spr.loc[formula_Spr["Z"] <= 0, "Y_pred_Springate"] = 1
Y_pred_formula_Spr=formula_Spr["Y_pred_Springate"]
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, Y_pred_formula_Spr)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, Y_pred_formula_Spr)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, Y_pred_formula_Spr)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, Y_pred_formula_Spr)

Accuracy: 0.3324451202409247
F1 score: 0.005673135066791271
Kappa: 0.0014263818200839218
Confusion matrix:


array([[  258,    31],
       [90408, 44781]], dtype=int64)

### 3.2) Valores Test de Grover

Su formula es:

Z = + 0.057 + 0.02 + 1.650 x Coef1 + 3.404 x Coef2 - 0.016 x Coef3

Z positivo empresa bien, Z negativo empresa mal.

Cogemos otro Dataframe, y multiplicamos primero y luego sumamos.

In [20]:
formula_Gro=X_test_SMOTE
formula_Gro=formula_Gro.drop(columns=["Coef1_Amat","Coef2_Amat","Coef3_Springate","Coef4_Springate","Coef4_Amat"])
formula_Gro.loc[:,'Coef1_Springate'] *= 1.65
formula_Gro.loc[:,'Coef2_Springate'] *= 3.404
formula_Gro.loc[:,'Coef3_Amat'] *= -0.016
formula_Gro["constant"]=0.077
formula_Gro["Z"] = formula_Gro.sum(axis=1)

formula_Gro["Y_pred_Grover"] = 2
formula_Gro.loc[formula_Gro["Z"] <= 0, "Y_pred_Grover"] = 1
Y_pred_formula_Gro=formula_Gro["Y_pred_Grover"]
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, Y_pred_formula_Gro)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, Y_pred_formula_Gro)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, Y_pred_formula_Gro)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, Y_pred_formula_Gro)

Accuracy: 0.7414857024756787
F1 score: 0.01050995903376183
Kappa: 0.00630509889578712
Confusion matrix:


array([[   186,    103],
       [ 34920, 100269]], dtype=int64)

### 3.3) Valores Test de Amat

Calculamos si está comprometida la situación de la empresa según Amat.

Su formula es:

Z = - 3.9 + 1.28 x Coef1 + 6.1 x Coef2 + 6.5 x Coef3 + 4.8 x Coef4

Z positivo empresa bien, Z negativo empresa mal.

Cogemos otro Dataframe, y multiplicamos primero y luego sumamos.

In [21]:
formula_Ama=X_test_SMOTE
formula_Ama=formula_Ama.drop(columns=["Coef1_Springate","Coef2_Springate","Coef3_Springate","Coef4_Springate"])
formula_Ama.loc[:,'Coef1_Amat'] *= 1.28
formula_Ama.loc[:,'Coef2_Amat'] *= 6.1
formula_Ama.loc[:,'Coef3_Amat'] *= 6.5
formula_Ama.loc[:,'Coef4_Amat'] *= 4.8
formula_Ama["constant"]=-3.9
formula_Ama["Z"] = formula_Ama.sum(axis=1)

formula_Ama["Y_pred_Amat"] = 2
formula_Ama.loc[formula_Ama["Z"] <= 0, "Y_pred_Amat"] = 1
Y_pred_formula_Ama=formula_Ama["Y_pred_Amat"]
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, Y_pred_formula_Ama)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, Y_pred_formula_Ama)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, Y_pred_formula_Ama)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, Y_pred_formula_Ama)

Accuracy: 0.5960303517914348
F1 score: 0.007399749714347897
Kappa: 0.0031691865927028218
Confusion matrix:


array([[  204,    85],
       [54644, 80545]], dtype=int64)

### 3.4) Combinado

In [22]:
labels_total=pd.concat([Y_test_SMOTE,Y_pred_formula_Spr, Y_pred_formula_Gro,Y_pred_formula_Ama], axis=1)
print(labels_total.to_string())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [23]:
bancarrota=labels_total.loc[(labels_total['Resultado']==1)]

In [24]:
at_least_1_bancarrota=sum(bancarrota.sum(axis=1)<7)
print(at_least_1_bancarrota)
all_match_bancarrota=sum(bancarrota.sum(axis=1)==4)
print(all_match_bancarrota)

272
152


### 4) Modelos ML

### 4.1) Estudio con Springate

In [25]:
X_train_SMOTE_Springate=X_train_SMOTE.drop(columns=["Coef1_Amat","Coef2_Amat","Coef3_Amat","Coef4_Amat"])
X_train_SMOTE_Springate

,Coef1_Springate,Coef2_Springate,Coef3_Springate,Coef4_Springate
0,-0.003900,0.023700,0.030900,0.981600
1,-0.036800,0.192000,0.124100,2.056700
2,0.206859,-0.122366,-0.351874,0.882378
3,0.056200,-0.017300,-0.112400,0.082200
4,-0.441470,-2.041330,-0.853690,0.485356
...,...,...,...,...
1081505,0.523600,-0.013600,-0.031800,0.940900
1081506,0.161500,0.103600,0.299100,1.730700
1081507,0.778800,0.002800,0.004000,1.459800
1081508,0.036000,0.014900,9.598900,0.038800


In [26]:
X_test_SMOTE_Springate=X_test_SMOTE.drop(columns=["Coef1_Amat","Coef2_Amat","Coef3_Amat","Coef4_Amat"])
X_test_SMOTE_Springate

,Coef1_Springate,Coef2_Springate,Coef3_Springate,Coef4_Springate
1156,0.4748,0.0758,0.2129,3.4304
1157,-0.1940,-0.1275,-0.4370,0.2235
1158,0.0978,0.0282,0.1630,0.4326
1159,0.0217,-0.0145,-0.6856,0.0002
1160,0.0953,-0.9579,-1.9177,2.7373
...,...,...,...,...
675939,0.1232,-0.1647,-5.9479,0.0000
675940,0.1989,-2.1463,-0.7942,0.0000
675941,-0.0927,0.0000,0.0208,0.0000
675942,0.6813,-0.0064,-0.0240,0.0000


K Nearest Neighbours

In [27]:
knn = KNeighborsClassifier()
knn.fit(X_train_SMOTE_Springate, Y_train_SMOTE)
y_pred = knn.predict(X_test_SMOTE_Springate)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [28]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.8472371898020343
F1 score: 0.008812260536398468
Kappa: 0.004624392388091869
Confusion matrix:


array([[    92,    197],
       [ 20499, 114690]], dtype=int64)

Random Forest

In [29]:
rf = RandomForestClassifier()
rf.fit(X_train_SMOTE_Springate, Y_train_SMOTE)
y_pred = rf.predict(X_test_SMOTE_Springate)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [30]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.932180870694873
F1 score: 0.011830501183050119
Kappa: 0.0077286816868509245
Confusion matrix:


array([[    55,    234],
       [  8954, 126235]], dtype=int64)

Decision Forest

In [31]:
clf = tree.DecisionTreeClassifier()
clf.fit(X_train_SMOTE_Springate, Y_train_SMOTE)
y_pred = clf.predict(X_test_SMOTE_Springate)

In [32]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.9209908619849717
F1 score: 0.010720887245841035
Kappa: 0.006595854679267266
Confusion matrix:


array([[    58,    231],
       [ 10473, 124716]], dtype=int64)

Naive Bayes

In [33]:
gnb = GaussianNB()
gnb.fit(X_train_SMOTE_Springate, Y_train_SMOTE)
y_pred = gnb.predict(X_test_SMOTE_Springate)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [34]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.006776007912723837
F1 score: 0.0041444641799881585
Kappa: -0.00011326829159230556
Confusion matrix:


array([[   280,      9],
       [134551,    638]], dtype=int64)

Multi-layer Perceptron Classifier Neural newtork

In [35]:
mlp = MLPClassifier()
mlp.fit(X_train_SMOTE_Springate, Y_train_SMOTE)
y_pred = mlp.predict(X_test_SMOTE_Springate)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [36]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.6486883479236482
F1 score: 0.007755331790606041
Kappa: 0.003529629239893639
Confusion matrix:


array([[  186,   103],
       [47492, 87697]], dtype=int64)

Linear Discriminant Analysis

In [37]:
lda = LinearDiscriminantAnalysis()
lda.fit(X_train_SMOTE_Springate, Y_train_SMOTE)
y_pred = lda.predict(X_test_SMOTE_Springate)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [38]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.6543719275454317
F1 score: 0.005183878986169242
Kappa: 0.0009477173388702242
Confusion matrix:


array([[  122,   167],
       [46658, 88531]], dtype=int64)

### 4.2) Estudio con Grover

In [39]:
X_train_SMOTE_Grover=X_train_SMOTE.drop(columns=["Coef1_Amat","Coef2_Amat","Coef3_Springate","Coef4_Springate","Coef4_Amat"])
X_train_SMOTE_Grover

,Coef1_Springate,Coef2_Springate,Coef3_Amat
0,-0.003900,0.023700,0.007300
1,-0.036800,0.192000,0.144000
2,0.206859,-0.122366,-0.114246
3,0.056200,-0.017300,-0.016200
4,-0.441470,-2.041330,-2.275014
...,...,...,...
1081505,0.523600,-0.013600,-0.015900
1081506,0.161500,0.103600,0.059900
1081507,0.778800,0.002800,0.001800
1081508,0.036000,0.014900,0.196600


In [40]:
X_test_SMOTE_Grover=X_test_SMOTE.drop(columns=["Coef1_Amat","Coef2_Amat","Coef3_Springate","Coef4_Springate","Coef4_Amat"])
X_test_SMOTE_Grover

,Coef1_Springate,Coef2_Springate,Coef3_Amat
1156,0.4748,0.0758,0.0568
1157,-0.1940,-0.1275,-0.1274
1158,0.0978,0.0282,0.0209
1159,0.0217,-0.0145,-0.0251
1160,0.0953,-0.9579,-0.8234
...,...,...,...
675939,0.1232,-0.1647,-0.1649
675940,0.1989,-2.1463,-1.6097
675941,-0.0927,0.0000,0.0117
675942,0.6813,-0.0064,0.0025


K Nearest Neighbours

In [41]:
knn = KNeighborsClassifier()
knn.fit(X_train_SMOTE_Grover, Y_train_SMOTE)
y_pred = knn.predict(X_test_SMOTE_Grover)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [42]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.7499520217304655
F1 score: 0.008371875182951818
Kappa: 0.004159186589036468
Confusion matrix:


array([[   143,    146],
       [ 33730, 101459]], dtype=int64)

Random Forest

In [43]:
rf = RandomForestClassifier()
rf.fit(X_train_SMOTE_Grover, Y_train_SMOTE)
y_pred = rf.predict(X_test_SMOTE_Grover)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [44]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.8194024121997667
F1 score: 0.01099478556125955
Kappa: 0.006806951588128718
Confusion matrix:


array([[   136,    153],
       [ 24314, 110875]], dtype=int64)

Decision Forest

In [45]:
clf = tree.DecisionTreeClassifier()
clf.fit(X_train_SMOTE_Grover, Y_train_SMOTE)
y_pred = clf.predict(X_test_SMOTE_Grover)

In [46]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.8122351968585305
F1 score: 0.009346522314822026
Kappa: 0.005149888111478318
Confusion matrix:


array([[   120,    169],
       [ 25269, 109920]], dtype=int64)

Naive Bayes

In [47]:
gnb = GaussianNB()
gnb.fit(X_train_SMOTE_Grover, Y_train_SMOTE)
y_pred = gnb.predict(X_test_SMOTE_Grover)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [48]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.00596406796675475
F1 score: 0.004082175977281804
Kappa: -0.00017582971835428118
Confusion matrix:


array([[   276,     13],
       [134657,    532]], dtype=int64)

Multi-layer Perceptron Classifier Neural newtork

In [49]:
mlp = MLPClassifier()
mlp.fit(X_train_SMOTE_Grover, Y_train_SMOTE)
y_pred = mlp.predict(X_test_SMOTE_Grover)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [50]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.6791065708085445
F1 score: 0.009071845368344275
Kappa: 0.004854147295253131
Confusion matrix:


array([[  199,    90],
       [43384, 91805]], dtype=int64)

Linear Discriminant Analysis

In [51]:
lda = LinearDiscriminantAnalysis()
lda.fit(X_train_SMOTE_Grover, Y_train_SMOTE)
y_pred = lda.predict(X_test_SMOTE_Grover)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [52]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.6774310220109538
F1 score: 0.005235482916391614
Kappa: 0.0010014182259732163
Confusion matrix:


array([[  115,   174],
       [43527, 91662]], dtype=int64)

### 4.3) Estudio con Amat

In [53]:
X_train_SMOTE_Amat=X_train_SMOTE.drop(columns=["Coef1_Springate","Coef2_Springate","Coef3_Springate","Coef4_Springate"])
X_train_SMOTE_Amat

,Coef1_Amat,Coef2_Amat,Coef3_Amat,Coef4_Amat
0,1.537100,-0.125000,0.007300,-0.058500
1,0.403700,-0.547600,0.144000,-0.263000
2,0.852863,0.398709,-0.114246,-0.291273
3,1.718300,0.732900,-0.016200,-0.022100
4,0.102403,-2.265369,-2.275014,0.995709
...,...,...,...,...
1081505,1.319600,0.124700,-0.015900,-0.127600
1081506,1.549800,0.301100,0.059900,0.198900
1081507,1.258300,0.299600,0.001800,0.006100
1081508,6.931500,0.814000,0.196600,0.241500


In [54]:
X_test_SMOTE_Amat=X_test_SMOTE.drop(columns=["Coef1_Springate","Coef2_Springate","Coef3_Springate","Coef4_Springate"])
X_test_SMOTE_Amat

,Coef1_Amat,Coef2_Amat,Coef3_Amat,Coef4_Amat
1156,2.8091,0.6440,0.0568,0.0883
1157,0.5615,0.0590,-0.1274,-2.1603
1158,1.5040,0.7844,0.0209,0.0267
1159,12.5930,0.3185,-0.0251,-0.0787
1160,0.1924,-0.6964,-0.8234,1.1824
...,...,...,...,...
675939,9.4479,0.9574,-0.1649,-0.1722
675940,0.1321,-1.7025,-1.6097,0.9455
675941,1.0615,0.4367,0.0117,0.0269
675942,3.6889,0.7320,0.0025,0.0034


K Nearest Neighbours

In [55]:
knn = KNeighborsClassifier()
knn.fit(X_train_SMOTE_Amat, Y_train_SMOTE)
y_pred = knn.predict(X_test_SMOTE_Amat)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [56]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.8507506753864096
F1 score: 0.010666405714844895
Kappa: 0.00648764564948856
Confusion matrix:


array([[   109,    180],
       [ 20040, 115149]], dtype=int64)

Random Forest

In [57]:
rf = RandomForestClassifier()
rf.fit(X_train_SMOTE_Amat, Y_train_SMOTE)
y_pred = rf.predict(X_test_SMOTE_Amat)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [58]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.9303576964525606
F1 score: 0.013178537809852526
Kappa: 0.009078684322106345
Confusion matrix:


array([[    63,    226],
       [  9209, 125980]], dtype=int64)

Decision Forest

In [59]:
clf = tree.DecisionTreeClassifier()
clf.fit(X_train_SMOTE_Amat, Y_train_SMOTE)
y_pred = clf.predict(X_test_SMOTE_Amat)

In [60]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.9261282274612852
F1 score: 0.011067193675889327
Kappa: 0.006951465155976555
Confusion matrix:


array([[    56,    233],
       [  9775, 125414]], dtype=int64)

Naive Bayes

In [61]:
gnb = GaussianNB()
gnb.fit(X_train_SMOTE_Amat, Y_train_SMOTE)
y_pred = gnb.predict(X_test_SMOTE_Amat)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [62]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.035607257266862515
F1 score: 0.004298190796994315
Kappa: 4.138807585152282e-05
Confusion matrix:


array([[   282,      7],
       [130647,   4542]], dtype=int64)

Multi-layer Perceptron Classifier Neural newtork

In [63]:
mlp = MLPClassifier()
mlp.fit(X_train_SMOTE_Amat, Y_train_SMOTE)
y_pred = mlp.predict(X_test_SMOTE_Amat)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [64]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.6209495268604497
F1 score: 0.00811233654607614
Kappa: 0.003886264066659928
Confusion matrix:


array([[  210,    79],
       [51274, 83915]], dtype=int64)

Linear Discriminant Analysis

In [65]:
lda = LinearDiscriminantAnalysis()
lda.fit(X_train_SMOTE_Amat, Y_train_SMOTE)
y_pred = lda.predict(X_test_SMOTE_Amat)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [66]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.4761068217718006
F1 score: 0.00493494840735756
Kappa: 0.0006887858610012954
Confusion matrix:


array([[  176,   113],
       [70863, 64326]], dtype=int64)

### 4.4) Estudio con todo

K Nearest Neighbours

In [67]:
knn = KNeighborsClassifier()
knn.fit(X_train_SMOTE, Y_train_SMOTE)
y_pred = knn.predict(X_test_SMOTE)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [68]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)


Accuracy: 0.9308743855090863
F1 score: 0.013067762672568236
Kappa: 0.008968422721866931
Confusion matrix:


array([[    62,    227],
       [  9138, 126051]], dtype=int64)

Random Forest

In [69]:
rf = RandomForestClassifier()
rf.fit(X_train_SMOTE, Y_train_SMOTE)
y_pred = rf.predict(X_test_SMOTE)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [70]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)


Accuracy: 0.9850603049941687
F1 score: 0.02598652550529355
Kappa: 0.022395760904084794
Confusion matrix:


array([[    27,    262],
       [  1762, 133427]], dtype=int64)

Decision Forest

In [71]:
clf = tree.DecisionTreeClassifier()
clf.fit(X_train_SMOTE, Y_train_SMOTE)
y_pred = clf.predict(X_test_SMOTE)

In [72]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.9657066091911601
F1 score: 0.01316907391673747
Kappa: 0.009201436498864068
Confusion matrix:


array([[    31,    258],
       [  4388, 130801]], dtype=int64)

Naive Bayes

In [73]:
gnb = GaussianNB()
gnb.fit(X_train_SMOTE, Y_train_SMOTE)
y_pred = gnb.predict(X_test_SMOTE)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [74]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.030831574130117067
F1 score: 0.004246896352977756
Kappa: -1.0171677732762063e-05
Confusion matrix:


array([[   280,      9],
       [131292,   3897]], dtype=int64)

Multi-layer Perceptron Classifier Neural newtork

In [75]:
mlp = MLPClassifier()
mlp.fit(X_train_SMOTE, Y_train_SMOTE)
y_pred = mlp.predict(X_test_SMOTE)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [76]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.77091483488094
F1 score: 0.010142246603304204
Kappa: 0.005940305792212941
Confusion matrix:


array([[   159,    130],
       [ 30906, 104283]], dtype=int64)

Linear Discriminant Analysis

In [77]:
lda = LinearDiscriminantAnalysis()
lda.fit(X_train_SMOTE, Y_train_SMOTE)
y_pred = lda.predict(X_test_SMOTE)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [78]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.7022690030853718
F1 score: 0.005669772716067643
Kappa: 0.0014398911952022697
Confusion matrix:


array([[  115,   174],
       [40162, 95027]], dtype=int64)